In [66]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import csv

In [67]:
model = gp.Model('Maritime Inventory Routing Problem')

# Creating classes in order organize the code

In [107]:
class Port:
    def __init__(self, number, berths):
        self.number = number
        self.berths = berths


class Node:
    def __init__(self, port, time):
        self.port = port
        self.time = time
        self.tuple = (port.number, time)
        self.incoming_arcs = set()
        self.outgoing_arcs = set()
        self.berths = port.berths
    
    def __repr__(self):
        return str(self.tuple)


class Arc:
    '''Lack distance'''
    '''Cost should be fixed to the arc for basic MIRP.
    In MIRPSO, the cost is a function of speed and distance.'''
    def __init__(self, origin_node, destination_node):
        self.origin_node = origin_node
        self.destination_node = destination_node
        self.tuple = (origin_node, destination_node)
        # self.cost = cost
    
    def __repr__(self):
        return str(self.origin_node) + ' -> ' + str(self.destination_node)

class Vessel:
    def __init__(self, max_inventory, initial_inventory):
        self.max_inventory = max_inventory
        self.inventory = initial_inventory
        

# Initial Parameters

All parameters should be set below

In [108]:
# Time periods
T = 3

# Number of vessels
V = 3

# Number of ports
P = 3

# Sets

Initializing sets

In [109]:
# Create a set of time periods from 1 to T.
time_period_range = set(range(1, T+1))

# Create a set of vessels from 1 to V
vessel_range = set(range(1, V+1))

# Create a set of ports from 1 to P
port_range = set(range(1, P+1))

ports = []
# Add source port
ports.append(Port(number=0, berths=V))
# For each port, create a Port object and add it to the set of ports
for i in port_range:
    ports.append(Port(number=i, berths=1))
# Add sink port
ports.append(Port(number=P+1, berths=V))

nodes = []
# Add source node
nodes.append(Node(port=ports[0], time=0))
# Create a set of regular nodes
regualar_ports = ports[1:-1]
nodes.extend(Node(port=j, time=t) for j in regualar_ports for t in time_period_range)
# Add sink nodes
nodes.append(Node(port=ports[P+1], time=T+1))


# Create a set of all incoming arcs to node n associated with vessel v in a time-space model
# There can only be an arc from a node to another node if the time period of the first node is less than the time period of the second node
def incoming_arcs(node, nodes):
    return {Arc(origin_node=i, destination_node=node) for i in nodes if i.time < node.time}

# Create a set of all outgoing arcs from node n associated with vessel v in a time-space model
# There can only be an arc from a node to another node if the time period of the first node is less than the time period of the second node
def outgoing_arcs(node, nodes):
    return {Arc(origin_node=node, destination_node=j) for j in nodes if node.time < j.time}

# Create the set of arcs associated with vessel v in a time-space model
# Note: The arcs need to be revised so that only feasible travels are included
def generate_all_arcs(nodes):
    arcs = set()
    
    # Arcs between all nodes
    for n in nodes:
        arcs.update(outgoing_arcs(n, nodes))
    return arcs

In [110]:
'''Need additional data to create the ports correctly in regards to the number of berths.
For now, we will assume that each port has one berth.'''

# Create the vessels. Homogenous fleet for now.
vessels = []
for v in vessel_range:
    vessel = Vessel(max_inventory=10, initial_inventory=0)
    vessels.append(vessel)
    
# Create the arcs
all_arcs = generate_all_arcs(nodes)

# Create the arcs for each node
for n in nodes:
    for a in all_arcs:
        if a.origin_node.tuple == n.tuple:
            n.outgoing_arcs.add(a)
        if a.destination_node.tuple == n.tuple:
            n.incoming_arcs.add(a)
            


{(1, 2) -> (3, 3), (1, 2) -> (4, 4), (1, 2) -> (2, 3), (1, 2) -> (1, 3)}


### All nodes and arcs have been created.


# Model parameters